In [1]:
import numpy as np
import pandas as pd
import pickle       # decompressing the pkl dataset
import matplotlib.pyplot as plt

In [ ]:
# df= pd.read_csv('./rxnpredictData/BHAmDataset_ControlRemoved.csv', header=0)
# AhnemanBHData= open('./data/AnhemanBuchwaldHartwig_ControlRemoved.pkl', 'wb')
# pickle.dump(df, AhnemanBHData)

# bh_pkl = open("./data/AnhemanBH_ControlRemoved.pkl",'rb')
# bh_contents= pickle.load(bh_pkl)
# xs=bh_contents.drop('yield', axis=1)
# ys=bh_contents['yield']

In [38]:
df= pd.read_csv('./rxnpredictData/BHAmDataset_ControlRemoved_Numbered.csv', header=0)
AhnemanBHData= open('./data/AnhemanBuchwaldHartwig_ControlRemoved_Numbered.pkl', 'wb')
pickle.dump(df, AhnemanBHData)

In [2]:
bh_pkl = open("./data/AnhemanBuchwaldHartwig_ControlRemoved_Numbered.pkl",'rb')
bh_contents= pickle.load(bh_pkl)
xs=bh_contents.drop('yield', axis=1)
ys=bh_contents['yield']

In [3]:
# Make a small dataset to play around with featurization
xysmall= bh_contents.sample(10)
xsmall= xysmall.drop('yield', axis=1)
ysmall= xysmall['yield']
print(xsmall.shape)
print(ysmall.shape)
xsmall.loc[:,'base_number']

(10, 10)
(10,)


894     0
162     1
1412    1
298     1
3409    0
3109    2
776     2
3323    0
335     1
2929    2
Name: base_number, dtype: int64

In [ ]:
# Strategy each csv to dictionary, replace smiles with dictionary, and then unpack w/ df.apply(pd.Series)

In [31]:
# PREPROCESSING - Don't run this again!
# Split the DFT data from Anheman et al. into individual csvs
additive= pd.read_csv('./rxnpredictData/additive.csv', header=0).transpose()
for column in additive.columns:
    filename= './rxnpredictData/additives/' + str(column+1) + '.csv'
    additive.loc[:,column].to_csv(filename, header=False)

base= pd.read_csv('./rxnpredictData/base.csv', header=0).transpose()
for column in base.columns:
    filename= './rxnpredictData/bases/' + str(column) + '.csv'
    base.loc[:,column].to_csv(filename, header=False)

ah= pd.read_csv('./rxnpredictData/aryl_halide.csv', header=0).transpose()
for column in ah.columns:
    filename= './rxnpredictData/aryl_halides/' + str(column+1) + '.csv'
    ah.loc[:,column].to_csv(filename, header=False)

lig= pd.read_csv('./rxnpredictData/ligand.csv', header=0).transpose()
for column in lig.columns:
    filename= './rxnpredictData/ligands/' + str(column) + '.csv'
    lig.loc[:,column].to_csv(filename, header=False)

In [10]:
# Fill in xs with features 
# Base
baseDicts = dict()
for idx in range(3):
    d = dict()
    f = open('./rxnpredictData/bases/' + str(idx) + '.csv')
    for line in f:
        line = line.strip('\n')
        (key, val) = line.split(",")
        d[key] = val
    baseDicts[idx] = d

xsmall['base_features']= xsmall.loc[:,'base_number'].map(baseDicts)

# Ligand
ligDicts = dict()
for idx in range(4):
    d = dict()
    f = open('./rxnpredictData/ligands/' + str(idx) + '.csv')
    for line in f:
        line = line.strip('\n')
        (key, val) = line.split(",")
        d[key] = val
    ligDicts[idx] = d

xsmall['ligand_features']= xsmall.loc[:,'ligand_number'].map(ligDicts)

# Aryl Halides
ahDicts = dict()
for idx in range(1,16):
    d = dict()
    f = open('./rxnpredictData/aryl_halides/' + str(idx) + '.csv')
    for line in f:
        line = line.strip('\n')
        (key, val) = line.split(",")
        d[key] = val
    ahDicts[idx] = d

xsmall['aryl_halide_features']= xsmall.loc[:,'aryl_halide_number'].map(ahDicts)

# Additives
addDicts = dict()
for idx in range(1,24):
    d = dict()
    f = open('./rxnpredictData/additives/' + str(idx) + '.csv')
    for line in f:
        line = line.strip('\n')
        (key, val) = line.split(",")
        d[key] = val
    addDicts[idx] = d

xsmall['additive_features']= xsmall.loc[:,'additive_number'].map(addDicts)
xsmall

,base_number,base_smiles,ligand_number,ligand_smiles,aryl_halide_number,aryl_halide_smiles,additive_number,additive,additive_smiles,product_smiles,base_features,ligand_features,aryl_halide_features,additive_features
894,0,CN1CCCN2CCCN=C12,1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=CC=CC=C2P(C(C...,11,Brc1ccccn1,2,4-phenylisoxazole,o1cc(cn1)c2ccccc2,Cc1ccc(Nc2ccccn2)cc1,"{'name': 'P2Et', 'base_*N1_electrostatic_charg...","{'name': 't-BuXPhos', 'ligand_*C10_NMR_shift':...","{'name': '2-bromopyridine', 'aryl_halide_*C1_N...","{'name': '4-phenylisoxazole', 'additive_*C3_NM..."
162,1,CN(C)/C(N(C)C)=N\C(C)(C)C,0,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,13,Clc1cccnc1,7,ethyl-5-methylisoxazole-3-carboxylate,CCOC(=O)c1cc(C)on1,Cc1ccc(Nc2cccnc2)cc1,"{'name': 'BTMG', 'base_*N1_electrostatic_charg...","{'name': 'XPhos', 'ligand_*C10_NMR_shift': '13...","{'name': '3-chloropyridine', 'aryl_halide_*C1_...",{'name': 'ethyl-5-methylisoxazole-3-carboxylat...
1412,1,CN(C)/C(N(C)C)=N\C(C)(C)C,0,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,8,CCc1ccc(Br)cc1,14,methyl-isoxazole-5-carboxylate,COC(=O)c1oncc1,CCc1ccc(Nc2ccc(C)cc2)cc1,"{'name': 'BTMG', 'base_*N1_electrostatic_charg...","{'name': 'XPhos', 'ligand_*C10_NMR_shift': '13...","{'name': '1-bromo-4-ethylbenzene', 'aryl_halid...","{'name': 'methyl-isoxazole-5-carboxylate', 'ad..."
298,1,CN(C)/C(N(C)C)=N\C(C)(C)C,1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=CC=CC=C2P(C(C...,14,Brc1cccnc1,5,ethyl-3-methylisoxazole-5-carboxylate,CCOC(=O)c1onc(C)c1,Cc1ccc(Nc2cccnc2)cc1,"{'name': 'BTMG', 'base_*N1_electrostatic_charg...","{'name': 't-BuXPhos', 'ligand_*C10_NMR_shift':...","{'name': '3-bromopyridine', 'aryl_halide_*C1_N...",{'name': 'ethyl-3-methylisoxazole-5-carboxylat...
3409,0,CN1CCCN2CCCN=C12,3,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,10,Clc1ccccn1,21,methyl-5-(furan-2-yl)isoxazole-3-carboxylate,COC(=O)c1cc(on1)c2occc2,Cc1ccc(Nc2ccccn2)cc1,"{'name': 'P2Et', 'base_*N1_electrostatic_charg...","{'name': 'AdBrettPhos', 'ligand_*C10_NMR_shift...","{'name': '2-chloropyridine', 'aryl_halide_*C1_...",{'name': 'methyl-5-(furan-2-yl)isoxazole-3-car...
3109,2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,2,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C(C)(C)C)...,10,Clc1ccccn1,17,3-methyl-5-phenylisoxazole,Cc1cc(on1)c2ccccc2,Cc1ccc(Nc2ccccn2)cc1,"{'name': 'MTBD', 'base_*N1_electrostatic_charg...","{'name': 't-BuBrettPhos', 'ligand_*C10_NMR_shi...","{'name': '2-chloropyridine', 'aryl_halide_*C1_...","{'name': '3-methyl-5-phenylisoxazole', 'additi..."
776,2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,0,CC(C)C1=CC(C(C)C)=CC(C(C)C)=C1C2=C(P(C3CCCCC3)...,12,Ic1ccccn1,4,3-phenylisoxazole,o1ccc(n1)c2ccccc2,Cc1ccc(Nc2ccccn2)cc1,"{'name': 'MTBD', 'base_*N1_electrostatic_charg...","{'name': 'XPhos', 'ligand_*C10_NMR_shift': '13...","{'name': '2-iodopyridine', 'aryl_halide_*C1_NM...","{'name': '3-phenylisoxazole', 'additive_*C3_NM..."
3323,0,CN1CCCN2CCCN=C12,3,CC(C1=C(C2=C(OC)C=CC(OC)=C2P(C34CC5CC(C4)CC(C5...,14,Brc1cccnc1,17,3-methyl-5-phenylisoxazole,Cc1cc(on1)c2ccccc2,Cc1ccc(Nc2cccnc2)cc1,"{'name': 'P2Et', 'base_*N1_electrostatic_charg...","{'name': 'AdBrettPhos', 'ligand_*C10_NMR_shift...","{'name': '3-bromopyridine', 'aryl_halide_*C1_N...","{'name': '3-methyl-5-phenylisoxazole', 'additi..."
335,1,CN(C)/C(N(C)C)=N\C(C)(C)C,1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=CC=CC=C2P(C(C...,6,COc1ccc(I)cc1,7,ethyl-5-methylisoxazole-3-carboxylate,CCOC(=O)c1cc(C)on1,COc1ccc(Nc2ccc(C)cc2)cc1,"{'name': 'BTMG', 'base_*N1_electrostatic_charg...","{'name': 't-BuXPhos', 'ligand_*C10_NMR_shift':...","{'name': '1-iodo-4-methoxybenzene', 'aryl_hali...",{'name': 'ethyl-5-methylisoxazole-3-carboxylat...
2929,2,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=CC=CC=C2P(C(C...,10,Clc1ccccn1,17,3-methyl-5-phenylisoxazole,Cc1cc(on1)c2ccccc2,Cc1ccc(Nc2ccccn2)cc1,"{'name': 'MTBD', 'base_*N1_electrostatic_charg...","{'name': 't-BuXPhos', 'ligand_*C10_NMR_shift':...","{'name': '2-chloropyridine', 'aryl_halide_*C1_...","{'name': '3-me

In [23]:
xsmallDFT= pd.concat([xsmall.base_features.apply(pd.Series), 
    xsmall.ligand_features.apply(pd.Series),
    xsmall.aryl_halide_features.apply(pd.Series),
    xsmall.additive_features.apply(pd.Series),], axis=1)
    
xsmallDFT= xsmallDFT.drop(labels= 'name', axis=1)
xsmallDFT

,base_*N1_electrostatic_charge,base_E_HOMO,base_E_LUMO,base_dipole_moment,base_electronegativity,base_hardness,base_molecular_volume,base_molecular_weight,base_ovality,base_surface_area,...,additive_E_LUMO,additive_V1_frequency,additive_V1_intensity,additive_dipole_moment,additive_electronegativity,additive_hardness,additive_molecular_volume,additive_molecular_weight,additive_ovality,additive_surface_area
894,-0.755,-0.1703,0.0569,5.039158,0.06,0.11,376.47,339.409,1.685,424.9,...,-0.0323,895.597,10.129,3.268545,0.13,0.1,154.39,145.161,1.225,170.5
162,-0.784,-0.2148,0.0029,1.126227,0.11,0.11,214.2,171.288,1.402,242.78,...,-0.0474,779.952,13.153,4.848261,0.16,0.11,155.89,155.153,1.318,184.63
1412,-0.784,-0.2148,0.0029,1.126227,0.11,0.11,214.2,171.288,1.402,242.78,...,-0.0655,909.75,2.153,4.564313,0.17,0.11,118.95,127.099,1.23,143.82
298,-0.784,-0.2148,0.0029,1.126227,0.11,0.11,214.2,171.288,1.402,242.78,...,-0.0589,1079.106,2.847,4.695918,0.17,0.11,155.9,155.153,1.319,184.82
3409,-0.755,-0.1703,0.0569,5.039158,0.06,0.11,376.47,339.409,1.685,424.9,...,-0.0633,764.902,31.415,3.85783,0.15,0.08,181.39,193.158,1.326,205.48
3109,-0.611,-0.1913,0.0505,2.703408,0.07,0.12,170.58,153.229,1.268,188.57,...,-0.0455,1074.375,4.339,2.980515,0.14,0.09,172.8,159.188,1.277,191.52
776,-0.611,-0.1913,0.0505,2.703408,0.07,0.12,170.58,153.229,1.268,188.57,...,-0.0385,1001.2,0.213,2.752454,0.14,0.1,154.42,145.161,1.226,170.65
3323,-0.755,-0.1703,0.0569,5.039158,0.06,0.11,376.47,339.409,1.685,424.9,...,-0.0455,1074.375,4.339,2.980515,0.14,0.09,172.8,159.188,1.277,191.52
335,-0.784,-0.2148,0.0029,1.126227,0.11,0.11,214.2,171.288,1.402,242.78,...,-0.0474,779.952,13.153,4.848261,0.16,0.11,155.89,155.153,1.318,184.63
2929,-0.611,-0.1913,0.0505,2.703408,0.07,0.12,170.58,153.229,1.268,188.57,...,-0.0455,1074.375,4.339,2.980515,0.14,0.09,172.8,159.188,1.277,191.52


In [29]:
# Fill in DFT features on X
xs['base_features']= xs.loc[:,'base_number'].map(baseDicts)
xs['ligand_features']= xs.loc[:,'ligand_number'].map(ligDicts)
xs['aryl_halide_features']= xs.loc[:,'aryl_halide_number'].map(ahDicts)
xs['additive_features']= xs.loc[:,'additive_number'].map(addDicts)

# Expand the DFT features and concatenate
xyDFT= pd.concat([xs.base_features.apply(pd.Series), 
    xs.ligand_features.apply(pd.Series),
    xs.aryl_halide_features.apply(pd.Series),
    xs.additive_features.apply(pd.Series),ys], axis=1)

# Drop the name columns
xyDFT= xyDFT.drop(labels= 'name', axis=1)

In [41]:
# Pickle the dataset
pickle.dump(xyDFT, open('./data/BH_CR_DFT.pkl', 'wb'))
xyDFT.to_csv('./data/BH_CR_DFT.csv')

In [40]:
xysmiles= pd.concat([xs.base_smiles, xs.ligand_smiles, xs.aryl_halide_smiles, xs.additive_smiles, ys], axis=1)

# Pickle the dataset
pickle.dump(xysmiles, open('./data/BH_CR_smiles.pkl', 'wb'))
xysmiles.to_csv('./data/BH_CR_smiles.csv')

In [39]:
xnumbers= pd.concat([xs.base_number, xs.ligand_number, xs.aryl_halide_number, xs.additive_number, ys], axis=1)

# Pickle the dataset
pickle.dump(xnumbers, open('./data/BH_CR_numbers.pkl', 'wb'))
xnumbers.to_csv('./data/BH_CR_numbers.csv')